In [ ]:
## cythonize at the first import
import mujoco_py

In [ ]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

# For later
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

In [ ]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
import matplotlib
matplotlib.use('Agg')

env = wrap_env(gym.make("Ant-v2"))

observation = env.reset()
for i in range(10):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
print('Loading video...')
show_video()

In [ ]:
#@title imports
import os
import time

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.mb_agent import MBAgent

%load_ext autoreload
%autoreload 2

In [ ]:
import ffmpeg

def vidwrite(fn, images, framerate=10, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,channel,height,width = images.shape
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate)
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        process.stdin.write(
            frame
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

## Run MBRL

In [ ]:
# Problem 1

!python ./run_hw4_mb.py --exp_name q1_cheetah_n500_arch1x32 \
    --env_name cheetah-cs285-v0 --add_sl_noise --n_iter 1 --batch_size_initial 20000 \
    --num_agent_train_steps_per_iter 500 --n_layers 1 --size 32 --scalar_log_freq -1 \
    --video_log_freq -1

In [ ]:
# Problem 2

!python ./run_hw4_mb.py --exp_name q2_obstacles_singleiteration \
    --env_name obstacles-cs285-v0 --add_sl_noise --num_agent_train_steps_per_iter 80 \
    --n_iter 1 --batch_size_initial 5000 --batch_size 200 --mpc_horizon 15 \
    --video_log_freq 1 -lr 0.005

In [ ]:
# Problem 3

!python ./run_hw4_mb.py --exp_name q3_obstacles --env_name obstacles-cs285-v0 \
    --add_sl_noise --num_agent_train_steps_per_iter 20 --batch_size_initial 5000 \
    --batch_size 200 --mpc_horizon 10 --n_iter 12 -lr 0.002 --video_log_freq 2

In [ ]:
import numpy as np

%cd "/root/Deep_RL_course/homework/hw4/data/hw4_q3_obstacles_obstacles-cs285-v0_28-11-2020_18-49-36"

array = np.load("train_rollouts_12.npy")
print(array.shape)
vidwrite("output.avi", array[0], framerate=10)

In [ ]:
# Problem 4

!python ./run_hw4_mb.py --exp_name q4_reacher_horizon5 --env_name reacher-cs285-v0 \
    --add_sl_noise --mpc_horizon 5 --num_agent_train_steps_per_iter 500 \
    --batch_size 500 --n_iter 1

In [ ]:
import numpy as np

%cd "/root/Deep_RL_course/homework/hw4/data/hw4_q3_obstacles_obstacles-cs285-v0_28-11-2020_18-49-36"

array = np.load("train_rollouts_12.npy")
print(array.shape)
vidwrite("output.avi", array[0], framerate=50)